In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from looptools.components import MokuPIDSymbolicController, MokuPIDController

In [ ]:
sps = 80e6
frfr = np.logspace(np.log10(1e-1), np.log10(1e6), int(1e5))
frfr = frfr[0:-1]

moku_pid = MokuPIDController("Moku PID", sps, -10, 1e3, None, 1e5, f_trans=1e2)
moku_pid_sym = MokuPIDSymbolicController("Moku PID Symbolic", sps, -10, 1e3, None, 1e5)

fig, ax = moku_pid.bode_plot(frfr, label='Numeric')
moku_pid_sym.bode_plot(frfr, axes=ax, label='Symbolic', ls='--')
plt.show()